In [1]:
# from scipy.misc import comb

In [2]:
def fallingFactorial(n,k):
    '''compute n x (n-1) x ... x (n-k + 1)'''
    return reduce(lambda x,y: x*y, (n - i for i in xrange(k)), 1)

In [3]:
def factorial(n):
    return fallingFactorial(n,n-1)

In [4]:
def binomial(n,k):
    return fallingFactorial(n,k)/factorial(k)

In [5]:
def multinomial(n, nlist):
    assert n == sum(nlist)
    return reduce(lambda x,y: x/y, map(factorial,nlist) ,factorial(n))

In [6]:
def divisors(n):
    return (x for x in xrange(1,n+1) if n%x == 0)

In [7]:
def memoize(f):
    '''Memoize a function
    Returns a version of f() which stores all computed values for later reference.
    The first time f(x) is called, we evaulate f(x) as usual. We then proceed to store
    the key-value-pair (x,f(x)) in a dictionary D. The next time f(x) is called, we notice
    that 'x' is in the keys of D and we just retrieve D[x] and return it rather than
    calling f(x) again.'''

    memory = {}

    def memoized_function(*args):
        try:
            return memory[args]
        except KeyError:
            f_x = f(*args)
            memory[args] = f_x
            return memory[args]

    return memoized_function

In [8]:
# def waysToGroupIntoIndistinctGroups(nElements,groupSizes):
#     ways_with_distinct_groups = multinomial(nElements,groupSizes)
#     groupSizeCounts = {}
#     for groupSize in groupSizes:
#         try:
#             groupSizeCounts[groupSize] += 1
#         except KeyError:
#             groupSizeCounts[groupSize] = 1
#     waysToRelabelGroups = reduce(lambda x,y: x * factorial(y), groupSizeCounts.values(), 1)
#     assert ways_with_distinct_groups % waysToRelabelGroups == 0
#     return ways_with_distinct_groups / waysToRelabelGroups
    

In [9]:
@memoize
def t_p_LU(n,l):
    '''count the number of planted unordered trees with n nodes, and l leaves, where
    leaves (nodes with no children) are labelled.'''
    if n==1 and l==1:
        return 1
    if n < 1:
        return 0
    if 2 > l or l > n-1:
        return 0
    else:
        a = t_p_LU(n - 1, l)
        b = t_np_LU(n-1,l-1)
        result = a + b
        return result

In [14]:
@memoize
def t_np_LU(n,l):
    if l == n-1:
        return 1
    if 2 <= l < n-1 and n > 2:
        summand = 0
        for n1 in xrange(2,n):
            for l1 in xrange(1,min(l,n1)):
                n2 = n - n1
                l2 = l - l1
                labelFactor = binomial(l,l1)
                t1Count = t_p_LU(n1,l1 + 1) + t_np_LU(n1,l1)
                t2Count = t_p_LU(n2,l2 + int(n2 > 1)) + t_np_LU(n2,l2)
                summand += labelFactor * n2 * t1Count * t2Count
        try:
            assert summand % (n - 1) == 0
        except AssertionError:
            print 'Count (= %i) is not divisible by n-1 (= %i)'%(summand,n-1)
            print ' Make sure the numebrs and boundary-conditions are right!'
        return summand / (n - 1)
    else:
        return 0

In [11]:
def generateStateSpaceTable(nMax = 20, sMax = 5, t_function = t_np_LU):
    header = '\\begin{tabular}{%s}'%('r'*(sMax + 2))
    header += '\n'
    header += 'Sample Size & \\multicolumn{%i}{c}{Segregating Sites}'%(sMax + 1)
    header += '\\\\\n'
    header += ' &' + ' &'.join([str(s) for s in range(0,sMax+1)])
    lines = [header]
    for n in range(2,nMax+1):
        line = '%i '%n
        for s in range(0,sMax+1):
            N = n + s + 1
            l = n
            states = t_function(N,l)
            line += '& {:,}'.format(states)
            # print '%i Lab. seq., %i Ulab. pos : %i'%(n,s,t_np_LU(N,l))
        lines.append(line)
    return '\\\\\n'.join(lines) + '\n\\end{tabular}'

print generateStateSpaceTable()

\begin{tabular}{rrrrrrr}
Sample Size & \multicolumn{6}{c}{Segregating Sites}\\
 &0 &1 &2 &3 &4 &5\\
2 & 1& 2& 3& 4& 5& 6\\
3 & 1& 6& 18& 40& 75& 126\\
4 & 1& 14& 75& 260& 700& 1,596\\
5 & 1& 30& 270& 1,400& 5,250& 15,876\\
6 & 1& 62& 903& 6,804& 34,755& 136,962\\
7 & 1& 126& 2,898& 31,080& 212,625& 1,076,922\\
8 & 1& 254& 9,075& 136,420& 1,233,650& 7,941,912\\
9 & 1& 510& 27,990& 583,000& 6,897,000& 55,927,872\\
10 & 1& 1,022& 85,503& 2,446,004& 37,542,505& 380,618,238\\
11 & 1& 2,046& 259,578& 10,130,120& 200,375,175& 2,524,159,638\\
12 & 1& 4,094& 784,875& 41,566,980& 1,053,834,600& 16,409,559,348\\
13 & 1& 8,190& 2,366,910& 169,423,800& 5,480,952,750& 105,034,499,388\\
14 & 1& 16,382& 7,125,303& 687,195,604& 28,263,758,255& 664,123,506,234\\
15 & 1& 32,766& 21,425,058& 2,777,349,160& 144,790,477,725& 4,158,489,610,674\\
16 & 1& 65,534& 64,373,475& 11,195,227,940& 737,946,423,550& 25,836,473,372,304\\
17 & 1& 131,070& 193,317,030& 45,038,667,800& 3,746,030,452,500& 159,514,076,776,82

In [12]:
# for n in range(2,11):
#     for s in range(0,11):
#         N = n + s + 1
#         l = n
#         print '%i Lab. seq., %i Ulab. pos : %i'%(n,s,t_np_LU(N,l))

In [13]:
## WRONG!
# def tLU(n,l):
#     if n < 1 or l > n:
#         return 0L
#     if n == 1:
#         return long(l == 1)
#     elif l == 1:
#         return 1
#     else:
#         summand = float(0L)
#         for n1 in range(1,n):
#             m = n - n1
#             n2Range = divisors(m)
#             # jRange = (m/d for d in dRange)
#             for n2 in n2Range:
#                 j = m/n2
#                 for l2 in range(1,min(n2,l - 1)):
#                     l1 = l - l2 * j
#                     assert l1 > 0
#                     t1 = tLU(n1,l1)
#                     t2 = tLU(n2,l2)
#                     weight = n2
                    
#                     waysToDistributeLabels = waysToGroupIntoIndistinctGroups(l,(l1,)+(l2,)*j)
#                     print t1,n1,l1
#                     print t2,n2,l2
#                     print weight
#                     print waysToDistributeLabels
#                     summand +=  t1 * t2 * weight * waysToDistributeLabels
#         print 'summand_final = %s'%str(summand)
#         return summand / (n - 1)